### 多输入多输出通道

In [1]:
# 实现多输入通道互相关运算
import torch
from d2l import torch as d2l


def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度(通道维度),再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [2]:
# 验证
# X是双通道输入(2*(3*3)), K是2个二维卷积核(2*(2*2))
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
X, K, corr2d_multi_in(X, K)

(tensor([[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]],
 
         [[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]),
 tensor([[[0., 1.],
          [2., 3.]],
 
         [[1., 2.],
          [3., 4.]]]),
 tensor([[ 56.,  72.],
         [104., 120.]]))

In [3]:
# 多输入,多输出通道
def corr2d_multi_in_out(X, K):
    # 在第0维,将列表中的tensor拼接为一个tensor
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)


# k: 3个输出通道,拼接在一起
K = torch.stack((K, K + 1, K + 2), 0)  # 不同的输出通道可以识别特定模式
# 输入X是2个通道, 输出是3个通道(每个通道两个核,对应处理2个输入通道)
K.shape, corr2d_multi_in_out(X, K)

(torch.Size([3, 2, 2, 2]),
 tensor([[[ 56.,  72.],
          [104., 120.]],
 
         [[ 76., 100.],
          [148., 172.]],
 
         [[ 96., 128.],
          [192., 224.]]]))

In [4]:
# 1×1卷积, 等价于全连接
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层计算
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))


X = torch.normal(0, 1, (3, 3, 3))  # (通道,高,宽)
K = torch.normal(0, 1, (2, 3, 1, 1))  # (输出通道,输入通道,核高,核宽)
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6